# Anti-UAV Drone Detection with YOLOv9

This notebook demonstrates real-time drone detection in videos using YOLOv9 fine-tuned on the Anti-UAV dataset.

## Features
- **Video Processing**: Detect drones frame-by-frame in video files
- **Annotated Output**: Generate videos with red bounding boxes around detected drones
- **JSON Predictions**: Export predictions in Anti-UAV format (visible.json)
- **Performance Metrics**: Track detection confidence and coverage


## Setup

In [ ]:
import sys
from pathlib import Path
import json
import cv2
import matplotlib.pyplot as plt
from IPython.display import Video, display

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / 'scripts'))
sys.path.insert(0, str(PROJECT_ROOT / 'models' / 'yolov9'))

from inference import DroneDetector

print("✓ Setup complete!")

## Configuration

In [ ]:
# Paths
WEIGHTS_PATH = PROJECT_ROOT / 'runs' / 'train' / 'quick-test' / 'weights' / 'best.pt'
VIDEO_PATH = PROJECT_ROOT / 'data' / 'subset' / '20190925_101846_1_2' / 'visible.mp4'
OUTPUT_DIR = PROJECT_ROOT / 'outputs'

# Detection parameters
CONF_THRESHOLD = 0.25  # Confidence threshold
IOU_THRESHOLD = 0.45   # NMS IoU threshold
DEVICE = '0'           # CUDA device ('0', '1', ...) or 'cpu'

print(f"Weights: {WEIGHTS_PATH}")
print(f"Video: {VIDEO_PATH}")
print(f"Output: {OUTPUT_DIR}")

## Load Model

In [ ]:
# Initialize detector
detector = DroneDetector(
    weights_path=str(WEIGHTS_PATH),
    device=DEVICE,
    conf_thres=CONF_THRESHOLD,
    iou_thres=IOU_THRESHOLD,
    img_size=640
)

## Process Video

In [ ]:
# Setup output paths
video_name = VIDEO_PATH.parent.name  # Use parent directory name (e.g., "20190925_101846_1_2")
output_video_path = OUTPUT_DIR / 'videos' / f"{video_name}_visible_detected.mp4"
output_json_path = OUTPUT_DIR / 'predictions' / f"{video_name}_visible.json"

output_video_path.parent.mkdir(parents=True, exist_ok=True)
output_json_path.parent.mkdir(parents=True, exist_ok=True)

# Run inference
results = detector.process_video(
    video_path=VIDEO_PATH,
    output_video_path=output_video_path,
    output_json_path=output_json_path,
    draw_boxes=True,
    box_color=(0, 0, 255),  # Red
    box_thickness=2
)

## Results Analysis

In [ ]:
# Load results
with open(output_json_path, 'r') as f:
    predictions = json.load(f)

total_frames = len(predictions['exist'])
detected_frames = sum(predictions['exist'])
detection_rate = detected_frames / total_frames * 100

print("="*60)
print("DETECTION RESULTS")
print("="*60)
print(f"Total frames: {total_frames}")
print(f"Frames with detections: {detected_frames}")
print(f"Detection rate: {detection_rate:.2f}%")
print("="*60)

## Visualize Sample Frames

In [ ]:
# Extract sample frames
cap = cv2.VideoCapture(str(output_video_path))

# Get frames at different timestamps
sample_frames_idx = [0, total_frames // 4, total_frames // 2, 3 * total_frames // 4]
sample_frames = []

for idx in sample_frames_idx:
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    ret, frame = cap.read()
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        sample_frames.append((idx, frame_rgb))

cap.release()

# Display
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for i, (idx, frame) in enumerate(sample_frames):
    axes[i].imshow(frame)
    axes[i].set_title(f'Frame {idx} - Detection: {"Yes" if predictions["exist"][idx] == 1 else "No"}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Display Output Video

**Note**: Video playback may not work in all Jupyter environments. If the video doesn't play, check the output file directly.

In [ ]:
# Display video (if supported)
try:
    display(Video(str(output_video_path), width=800))
except Exception as e:
    print(f"Video display not supported in this environment: {e}")
    print(f"\nOutput video saved at: {output_video_path}")
    print(f"Open it with any video player to view the results.")

## Detection Timeline

In [ ]:
# Plot detection timeline
plt.figure(figsize=(15, 3))
plt.plot(predictions['exist'], linewidth=0.5)
plt.fill_between(range(len(predictions['exist'])), predictions['exist'], alpha=0.3)
plt.xlabel('Frame Number')
plt.ylabel('Detection (1=Yes, 0=No)')
plt.title('Drone Detection Timeline')
plt.grid(True, alpha=0.3)
plt.ylim(-0.1, 1.1)
plt.tight_layout()
plt.show()

## Bounding Box Statistics

In [ ]:
# Analyze bounding box sizes
box_areas = []
for exist, rect in zip(predictions['exist'], predictions['gt_rect']):
    if exist == 1:
        x, y, w, h = rect
        area = w * h
        box_areas.append(area)

if box_areas:
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(box_areas, bins=30, edgecolor='black')
    plt.xlabel('Bounding Box Area (pixels)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Detection Sizes')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(box_areas, linewidth=1)
    plt.xlabel('Detection Index')
    plt.ylabel('Bounding Box Area (pixels)')
    plt.title('Detection Size Over Time')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Average box area: {sum(box_areas)/len(box_areas):.2f} pixels")
    print(f"Min box area: {min(box_areas):.2f} pixels")
    print(f"Max box area: {max(box_areas):.2f} pixels")
else:
    print("No detections found in video.")

## Export Summary

In [ ]:
summary = {
    "video": str(VIDEO_PATH.name),
    "total_frames": total_frames,
    "detected_frames": detected_frames,
    "detection_rate": f"{detection_rate:.2f}%",
    "avg_box_area": f"{sum(box_areas)/len(box_areas):.2f}" if box_areas else "N/A",
    "output_video": str(output_video_path),
    "output_json": str(output_json_path)
}

print(json.dumps(summary, indent=2))